In [134]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [165]:
from src.utils import (
    load_data,
    remove_duplicates_in_coloumn,
    convert_from_degree_to_ciruclar,
    resample_hourly,
    create_time_features,
    find_repeated_indices,
    load_val_dates,
    create_lag_features
)

import pandas as pd
import numpy as np

In [136]:
data = load_data()

In [137]:
# Remove duplicate rows
for location in data.keys():
    df = data[location]

    df["y"] = remove_duplicates_in_coloumn(df["y"], "time")
    df["X_test_estimated"] = remove_duplicates_in_coloumn(df["X_test_estimated"], "date_forecast")
    df["X_train_estimated"] = remove_duplicates_in_coloumn(df["X_train_estimated"], "date_forecast")
    df["X_train_observed"] = remove_duplicates_in_coloumn(df["X_train_observed"], "date_forecast")

    data[location] = df

In [138]:
# Drop columns
columns_to_drop = [
    "ceiling_height_agl:m",
    "cloud_base_agl:m",
    "snow_density:kgm3",
    "elevation:m",
    "precip_5min:mm",
    "precip_type_5min:idx",
    "pressure_50m:hPa",
    "snow_drift:idx",
    "wind_speed_u_10m:ms",
    "wind_speed_v_10m:ms",
    "wind_speed_w_1000hPa:ms",
    "date_calc",

    # Duplicate columns
    "diffuse_rad_1h:J",
    "direct_rad_1h:J",
    "clear_sky_energy_1h:J",
]

for location in data.keys():
    df = data[location]

    df["X_test_estimated"] = df["X_test_estimated"].drop(columns=columns_to_drop, errors="ignore")
    df["X_train_estimated"] = df["X_train_estimated"].drop(columns=columns_to_drop, errors="ignore")
    df["X_train_observed"] = df["X_train_observed"].drop(columns=columns_to_drop, errors="ignore")

    data[location] = df

In [139]:
# Sun azimuth feature engineering
for location in data.keys():
    df = data[location]

    df["X_test_estimated"] = convert_from_degree_to_ciruclar(df["X_test_estimated"], "sun_azimuth:d")
    df["X_train_estimated"] = convert_from_degree_to_ciruclar(df["X_train_estimated"], "sun_azimuth:d")
    df["X_train_observed"] = convert_from_degree_to_ciruclar(df["X_train_observed"], "sun_azimuth:d")

    data[location] = df

In [140]:
# Reduce granularity of data to hourly
for location in data.keys():
    df = data[location]

    df["X_test_estimated"] = resample_hourly(df["X_test_estimated"], func="sum")
    df["X_train_estimated"] = resample_hourly(df["X_train_estimated"], func="sum")
    df["X_train_observed"] = resample_hourly(df["X_train_observed"], func="sum")

    data[location] = df

In [141]:
data["A"]["X_test_estimated"].columns

Index(['date_forecast', 'absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'clear_sky_rad:W', 'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'direct_rad:W', 'effective_cloud_cover:p', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'pressure_100m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'sun_azimuth:d_sin', 'sun_azimuth:d_cos'],
      dtype='object')

In [142]:
# Create lag features
for location in data.keys():
    df = data[location]

    df["X_test_estimated"] = create_lag_features(
        df["X_test_estimated"],
        columns=["effective_cloud_cover:p", "absolute_humidity_2m:gm3"],
        lag=-1
    )
    df["X_train_estimated"] = create_lag_features(
        df["X_train_estimated"],
        columns=["effective_cloud_cover:p", "absolute_humidity_2m:gm3"],
        lag=-1
    )
    df["X_train_observed"] = create_lag_features(
        df["X_train_observed"],
        columns=["effective_cloud_cover:p", "absolute_humidity_2m:gm3"],
        lag=-1
    )

    df["X_test_estimated"] = create_lag_features(
        df["X_test_estimated"],
        columns=["effective_cloud_cover:p", "absolute_humidity_2m:gm3"],
        lag=-2
    )
    df["X_train_estimated"] = create_lag_features(
        df["X_train_estimated"],
        columns=["effective_cloud_cover:p", "absolute_humidity_2m:gm3"],
        lag=-2
    )
    df["X_train_observed"] = create_lag_features(
        df["X_train_observed"],
        columns=["effective_cloud_cover:p", "absolute_humidity_2m:gm3"],
        lag=-2
    )

    data[location] = df

In [143]:
# Function that returns train_targets, observed and estimated sets left after filtering away NaN
def drop_nan_rows_in_target_and_train(df):
    df["y"] = df["y"].dropna(subset=["pv_measurement"])
    valid_dates = df["y"]["time"]

    df["X_train_observed"] = df["X_train_observed"][
        df["X_train_observed"]["date_forecast"].isin(valid_dates)
    ]
    df["X_train_estimated"] = df["X_train_estimated"][
        df["X_train_estimated"]["date_forecast"].isin(valid_dates)
    ]
    df["y"] = df["y"][
        df["y"]["time"].isin(df["X_train_observed"]["date_forecast"])
        | df["y"]["time"].isin(df["X_train_estimated"]["date_forecast"])
    ]

    return df

In [144]:
for location in data.keys():
    df = data[location]

    df = drop_nan_rows_in_target_and_train(df)

    data[location] = df

In [145]:
# Create time features
for location in data.keys():
    df = data[location]

    df["X_test_estimated"] = create_time_features(df["X_test_estimated"], "date_forecast")
    df["X_train_estimated"] = create_time_features(df["X_train_estimated"], "date_forecast")
    df["X_train_observed"] = create_time_features(df["X_train_observed"], "date_forecast")

    data[location] = df

In [146]:
print("A:")
print(data["A"]["X_train_estimated"].shape, data["A"]["X_train_observed"].shape, data["A"]["y"].shape)

print("B:")
print(data["B"]["X_train_estimated"].shape, data["B"]["X_train_observed"].shape, data["B"]["y"].shape)

print("C:")
print(data["C"]["X_train_estimated"].shape, data["C"]["X_train_observed"].shape, data["C"]["y"].shape)


A:
(4418, 44) (29667, 44) (34085, 2)
B:
(3625, 44) (29218, 44) (32843, 2)
C:
(2954, 44) (23141, 44) (26095, 2)


In [147]:
# Making training and validation data for A

X_train = pd.DataFrame()
y_train = pd.DataFrame()

X_validate = pd.DataFrame()
y_validate = pd.DataFrame()

for location in data.keys():
    percent_observed_train_a = 1
    percent_estimated_train_a = 1

    split_index_obs = int(
        len(data[location]["X_train_observed"]) * percent_observed_train_a
    )
    split_index_est = int(
        len(data[location]["X_train_estimated"]) * percent_estimated_train_a
    )

    X_train_observed_first_75 = data[location]["X_train_observed"][:split_index_obs]
    X_train_observed_last_25 = data[location]["X_train_observed"][split_index_obs:]

    X_train_estimated_first_75 = data[location]["X_train_estimated"][:split_index_est]
    X_train_estimated_last_25 = data[location]["X_train_estimated"][split_index_est:]

    X_train_loc = pd.concat([X_train_observed_first_75, X_train_estimated_first_75])
    y_train_loc = data[location]["y"][
        data[location]["y"]["time"].isin(X_train_loc["date_forecast"])
    ]

    X_validate_loc = pd.concat([X_train_observed_last_25, X_train_estimated_last_25])
    y_validate_loc = data[location]["y"][
        data[location]["y"]["time"].isin(X_validate_loc["date_forecast"])
    ]

    repeated_indices = find_repeated_indices(y_train_loc, "pv_measurement", 24)
    y_train_loc = y_train_loc.reset_index()
    y_train_loc = y_train_loc.drop(repeated_indices)
    X_train_loc = X_train_loc[X_train_loc["date_forecast"].isin(y_train_loc["time"])]

    repeated_indices = find_repeated_indices(y_validate_loc, "pv_measurement", 24)
    y_validate_loc = y_validate_loc.reset_index()
    y_validate_loc = y_validate_loc.drop(repeated_indices)
    X_validate_loc = X_validate_loc[
        X_validate_loc["date_forecast"].isin(y_validate_loc["time"])
    ]

    y_train_loc.reset_index(drop=True, inplace=True)
    X_train_loc.reset_index(drop=True, inplace=True)
    y_validate_loc.reset_index(drop=True, inplace=True)
    X_validate_loc.reset_index(drop=True, inplace=True)

    X_train_loc["location"] = location
    y_train_loc["location"] = location
    X_validate_loc["location"] = location
    y_validate_loc["location"] = location

    X_train_loc.drop("date_forecast", axis=1, inplace=True)
    y_train_loc.drop("time", axis=1, inplace=True)
    X_validate_loc.drop("date_forecast", axis=1, inplace=True)
    y_validate_loc.drop("time", axis=1, inplace=True)

    X_train_loc = X_train_loc.reset_index().drop(columns="index")
    one_hot = pd.get_dummies(X_train_loc["location"]).astype(int)
    X_train_loc = X_train_loc.drop("location", axis=1)
    X_train_loc = pd.merge(X_train_loc, one_hot, left_index=True, right_index=True)

    X_train = pd.concat([X_train_loc, X_train])
    y_train = pd.concat([y_train_loc, y_train])
    X_validate = pd.concat([X_validate_loc, X_validate])
    y_validate = pd.concat([y_validate_loc, y_validate])

In [148]:
data["A"]["X_test_estimated"]["location"] = "A"
data["B"]["X_test_estimated"]["location"] = "B"
data["C"]["X_test_estimated"]["location"] = "C"

X_test = pd.concat([data["A"]["X_test_estimated"], data["B"]["X_test_estimated"], data["C"]["X_test_estimated"]])
# filtering out invalid dates:
X_test = X_test[X_test["date_forecast"].isin(load_val_dates())]
# removing forecast coloum
X_test = X_test.drop("date_forecast", axis=1)

X_test = X_test.reset_index().drop(columns="index")
one_hot = pd.get_dummies(X_test["location"]).astype(int)
X_test = X_test.drop("location", axis=1)
X_test = pd.merge(X_test, one_hot, left_index=True, right_index=True)

In [149]:
columns_to_exclude = ["A", "B", "C", "dew_or_rime:idx", "is_day:idx", "_in_shadow:idx"]

columns_to_normalize = [col for col in X_train.columns if col not in columns_to_exclude]

#Min-max
# Calculate min and max values for scaling
X_min = X_train[columns_to_normalize].min()
X_max = X_train[columns_to_normalize].max()

# Apply min-max scaling to the columns to be normalized
X_train[columns_to_normalize] = (X_train[columns_to_normalize] - X_min) / (X_max - X_min)
X_validate[columns_to_normalize] = (X_validate[columns_to_normalize] - X_min) / (X_max - X_min)
X_test[columns_to_normalize] = (X_test[columns_to_normalize] - X_min) / (X_max - X_min)


In [150]:
from sklearn.preprocessing import MinMaxScaler

y_scaler = MinMaxScaler()
y_train["pv_measurement"] = y_scaler.fit_transform(y_train["pv_measurement"].values.reshape(-1,1))

In [151]:
X_train = X_train.reset_index().drop(columns="index")
y_train = y_train.reset_index().drop(columns="index")
new_train = pd.merge(X_train, y_train["pv_measurement"], left_index=True, right_index=True)
new_train = new_train.fillna(0)

X_validate = X_validate.reset_index().drop(columns="index")
y_validate = y_validate.reset_index().drop(columns="index")
new_validate = pd.merge(X_validate, y_validate["pv_measurement"], left_index=True, right_index=True)

In [152]:
new_train.to_csv("../data/processed/train.csv", index=False)
X_test.to_csv("../data/processed/X_test.csv", index=False)


In [153]:
new_train

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,direct_rad:W,effective_cloud_cover:p,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,pressure_100m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_melt_10min:mm,snow_water:kgm2,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,sun_azimuth:d_sin,sun_azimuth:d_cos,effective_cloud_cover:p-1,absolute_humidity_2m:gm3-1,effective_cloud_cover:p-2,absolute_humidity_2m:gm3-2,hour,dayofmonth,dayofweek,quarter,month,year,dayofyear,C,B,A,pv_measurement
0,0.381844,0.855916,0.504069,0.0,0.947467,0.203062,0.494805,0.15575,0.0,0.0,0.0,0.0,0.0,4.0,0.00,0.966593,0.963793,0.0,0.000000,0.60650,0.963424,0.0,0.0,0.000000,0.757786,0.000000,0.935614,0.31200,0.656377,0.094162,0.880756,0.176445,0.43100,0.361671,0.47575,0.340058,0.347826,0.100000,0.333333,0.666667,0.727273,0.0,0.673973,1.0,0.0,0.0,0.023930
1,0.361671,0.851359,0.608060,0.0,0.944828,0.228902,0.594162,0.43100,0.0,0.0,0.0,0.0,0.0,4.0,0.00,0.965516,0.962847,0.0,0.000000,0.53300,0.962418,0.0,0.0,0.000000,0.802339,0.000000,0.939159,0.86250,0.692945,0.112994,0.775686,0.083521,0.47575,0.340058,0.50000,0.338617,0.391304,0.100000,0.333333,0.666667,0.727273,0.0,0.673973,1.0,0.0,0.0,0.000000
2,0.481268,0.860123,0.584126,0.0,0.958280,0.230620,0.000768,1.00000,0.0,0.0,0.0,0.0,0.0,4.0,0.00,0.959002,0.956421,0.0,0.068182,0.88700,0.956191,0.0,0.0,0.053097,0.791845,0.072727,0.931245,1.00000,0.035227,0.406780,0.203736,0.097843,0.99900,0.471182,0.98725,0.461095,0.565217,0.133333,0.500000,0.666667,0.727273,0.0,0.676712,1.0,0.0,0.0,0.017093
3,0.471182,0.859246,0.473942,0.0,0.957258,0.279238,0.008889,0.99900,0.0,0.0,0.0,0.0,0.0,4.0,0.00,0.959649,0.957075,0.0,0.000000,0.89000,0.956789,0.0,0.0,0.035398,0.744924,0.072727,0.931245,1.00000,0.055717,0.406780,0.104731,0.194280,0.98725,0.461095,0.93025,0.453890,0.608696,0.133333,0.500000,0.666667,0.727273,0.0,0.676712,1.0,0.0,0.0,0.020511
4,0.461095,0.859597,0.336145,0.0,0.956152,0.253622,0.015767,0.98725,0.0,0.0,0.0,0.0,0.0,4.0,0.00,0.960295,0.957657,0.0,0.000000,0.87250,0.957388,0.0,0.0,0.035398,0.686971,0.072727,0.931162,0.99200,0.057336,0.399247,0.039536,0.305486,0.93025,0.453890,0.85800,0.449568,0.652174,0.133333,0.500000,0.666667,0.727273,0.0,0.676712,1.0,0.0,0.0,0.029058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81161,0.262248,0.895004,0.005056,0.0,0.927799,0.008439,0.000146,0.96700,0.0,0.0,0.0,0.0,0.0,2.0,0.75,0.972029,0.971577,0.0,0.000000,0.74625,0.971568,0.0,0.0,0.000000,0.498081,0.000000,0.906925,0.96700,0.310204,0.389831,0.090122,0.786391,0.94225,0.259366,0.94325,0.259366,0.826087,0.966667,1.000000,0.333333,0.272727,1.0,0.326027,0.0,0.0,1.0,0.001573
81162,0.259366,0.897283,0.000000,0.0,0.927373,0.000000,0.000000,0.94225,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.971981,0.971553,0.0,0.000000,0.76875,0.971568,0.0,0.0,0.000000,0.448059,0.000000,0.905688,0.94525,0.279298,0.350282,0.169562,0.875198,0.94325,0.259366,0.97775,0.259366,0.869565,0.966667,1.000000,0.333333,0.272727,1.0,0.326027,0.0,0.0,1.0,0.000000
81163,0.259366,0.898160,0.000000,0.0,0.927373,0.000000,0.000000,0.94325,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.971838,0.971408,0.0,0.000000,0.77775,0.971448,0.0,0.0,0.000000,0.410260,0.000000,0.905276,0.95675,0.275436,0.335217,0.271412,0.944571,0.97775,0.259366,0.98425,0.259366,0.913043,0.966667,1.000000,0.333333,0.272727,1.0,0.326027,0.0,0.0,1.0,0.000000
81164,0.259366,0.898160,0.000000,0.0,0.927373,0.000000,0.000000,0.97775,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.971694,0.971262,0.0,0.000000,0.79000,0.971329,0.0,0.0,0.004425,0.387405,0.072727,0.904617,0.98875,0.187551,0.308851,0.390789,0.987767,0.98425,0.259366,0.98425,0.259366,0.956522,0.96

In [160]:
predictions = pd.read_csv("../data/results/predictions.csv").drop(columns="Unnamed: 0")

In [163]:
prediction_df_scaled = y_scaler.inverse_transform(predictions)
prediction_df_scaled_df = pd.DataFrame(prediction_df_scaled)

In [166]:
resultframe = pd.DataFrame(columns = ["id", "prediction"])
resultframe["prediction"] = prediction_df_scaled_df
resultframe['prediction'] = np.where(resultframe['prediction'] < 0, 0, resultframe['prediction'])
resultframe["id"] = range(len(resultframe))
resultframe.head()

,id,prediction
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,84.553950
4,4,428.989739


In [169]:
import datetime


resultframe.to_csv("../data/results/"+ str(datetime.datetime.now()) + "-submission.csv", index=False)